In [2]:
# Let's start by retrieving some Magic card images using the Scryfall API
# https://scryfall.com/docs/api
# We'll be using the Scryfall python library
# https://github.com/NandaScott/Scrython

In [40]:
import requests
import json
import pprint
import time
import shutil
import os.path

URL='https://api.scryfall.com'
OUTDIR='/Users/agussman/Documents/DeepLens/deeplens-magiccards/data'


In [ ]:
r = requests.get('https://api.scryfall.com/cards/search?order=set&q=e:dom')

In [45]:
results = r.json()['data']
for result in results[:10]:
    #card = result
    #if 'all_parts' in result
    #print(result['image_uris']['png'])
    time.sleep(0.25)
    image_url = result['image_uris']['png']
    filename = "{}_{num:03d}.png".format(result['set'], num=int(result['collector_number']))
    filename = os.path.join(OUTDIR, result['set'], filename)
    print(filename)
    print("{} -> {}".format(image_url, filename))
    
    r = requests.get(image_url, stream=True)
    
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
    
    '''
    if r.status_code == 200:
        with open(os.path.join(OUTDIR, result['set'], filename), 'wb') as out_file:
            shutil.copyfileobj(response.raw(), out_file)
    else:
        print("Nope. {}".format(response.status_code))
    '''
    
    
#pprint.pprint(results[0:2])

/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_001.png
https://img.scryfall.com/cards/png/en/dom/1.png?1524789999 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_001.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_002.png
https://img.scryfall.com/cards/png/en/dom/2.png?1525862131 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_002.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_003.png
https://img.scryfall.com/cards/png/en/dom/3.png?1525862133 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_003.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_004.png
https://img.scryfall.com/cards/png/en/dom/4.png?1524790027 -> /Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_004.png
/Users/agussman/Documents/DeepLens/deeplens-magiccards/data/dom/dom_005.png
https://img.scryfall.com/cards/png/en/dom/5.png?1528770903 -> /Users

In [44]:
response.raw()

TypeError: 'HTTPResponse' object is not callable

In [4]:
import time

query = input("Type the name of the set: ")

time.sleep(0.05)
sets = scrython.sets.Sets()

for i in range(sets.data_length()):
    if sets.set_name(i) == query:
        print("Set code:", sets.set_code(i).upper())
        break
    else:
        continue

Type the name of the set: dom


RuntimeError: Cannot run the event loop while another loop is running